# Fake News Detection System using Lstm using new data set



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re

import seaborn as sns

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords



In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

# **Connecting To drive**

# unziping and reading the the csv files for data Preprocessing

In [4]:
train = pd.read_csv('D:\\Google\\Models\\fakeNewsDataset\\train.csv')
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
train['text'].fillna( train['title'],inplace=True)
train.isnull().sum()

id           0
title      558
author    1957
text         0
label        0
dtype: int64

In [7]:
X_train=train.drop('label',axis=1)
y_train=train['label']

# Cleaning the data (removing special characters)

# Gensim's Word2vec and similarity function used

In [8]:
y=train['label'].values

In [9]:
x = [d.split() for d in train['text'].tolist()]

In [10]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(x)

In [11]:
x=tokenizer.texts_to_sequences(x)

In [12]:
import gensim

In [13]:
from gensim.models import KeyedVectors

import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))


['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [14]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load('D:\\Google\\Models\\w2vgoogle\\w2v_model.bin')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Google\\Models\\w2vgoogle\\w2v_model.bin'

In [14]:
word2vec_vectors = gensim.downloader.load('word2vec-google-news-300')

In [17]:
word2vec_vectors.save('D:\\Google\\Models\\w2vgoogle\\w2v_model.bin')


In [27]:
word2vec_vectors.save('D:\\Google\\Models\\w2vgoogle\\w2v_model.kv')

In [72]:
word2vec_vectors

In [73]:
embedding_dimension = word2vec_vectors.vector_size

In [74]:
embedding_dimension

300

In [75]:
maxlen=1000
X=pad_sequences(x,maxlen=maxlen)

In [76]:
vocab_size = len(word2vec_vectors.key_to_index)


In [78]:
vocab_size

3000000

In [77]:
vocab=tokenizer.word_index


In [29]:
vocab_size

3000000

In [31]:
len(word2vec_vectors.vectors)

3000000

In [92]:
import numpy as np
from tqdm import tqdm

# Reduce vocabulary size
max_vocab_size = 500000
filtered_word_index = {word: idx for idx, (word, _) in enumerate(tokenizer.word_index.items()) if idx < max_vocab_size}

# Use smaller embedding dimension
embedding_dim = 100  # Adjust this as needed
embedding_matrix = np.zeros((max_vocab_size, embedding_dim), dtype=np.float32)  

# Iterate over the filtered word index and populate the embedding matrix
for word, i in tqdm(filtered_word_index.items()):
    try:
        embedding_value = word2vec_vectors[word]
        if embedding_value is not None:
            embedding_matrix[i] = embedding_value[:embedding_dim]  # Take only the first embedding_dim dimensions
    except KeyError:
        embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), embedding_dim)

# Now you can use embedding_matrix in your Keras model


100%|███████████████████████████████████████████████████████████████████████| 437459/437459 [00:38<00:00, 11443.76it/s]


In [96]:
len(embedding_matrix)

500000

In [94]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

In [114]:
help(Embedding)


Help on class Embedding in module keras.src.layers.core.embedding:

class Embedding(keras.src.layers.layer.Layer)
 |  Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, embeddings_constraint=None, mask_zero=False, **kwargs)
 |  
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  
 |  e.g. `[[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]`
 |  
 |  This layer can only be used on positive integer inputs of a fixed range.
 |  
 |  Example:
 |  
 |  >>> model = keras.Sequential()
 |  >>> model.add(keras.layers.Embedding(1000, 64, input_length=10))
 |  >>> # The model will take as input an integer matrix of size (batch,
 |  >>> # input_length), and the largest integer (i.e. word index) in the input
 |  >>> # should be no larger than 999 (vocabulary size).
 |  >>> # Now model.output_shape is (None, 10, 64), where `None` is the batch
 |  >>> # dimension.
 |  >>> input_array = np.random.randint(1000, size=(32, 10))
 |  >>> model.co

In [116]:
embedding_matrix.shape

(500000, 100)

In [125]:
# Retrieve the weights of the Embedding layer
embedding_weights = model.layers[0].get_weights()

# Print the shape of the weights
print("Shape of embedding weights:", embedding_weights[0].shape)


IndexError: list index out of range

In [128]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np

# Assuming you have defined max_vocab_size, embedding_dim, and embedding_matrix
model = Sequential()

# Initialize the Embedding layer without input_length and weights
embedding_layer = Embedding(input_dim=max_vocab_size, output_dim=100, trainable=False)

# Add the embedding layer to the model
model.add(embedding_layer)

# Set the weights of the embedding layer
model.layers[0].set_weights([embedding_matrix])

# Add LSTM and Dense layers
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


ValueError: You called `set_weights(weights)` on layer 'embedding_41' with a weight list of length 500000, but the layer was expecting 0 weights.

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot2.png', show_shapes=True, show_layer_names=True)